In [1]:
import pandas as pd
#import bson


In [2]:
#with open('emails.bson','rb') as f:
#    data = bson.decode_all(f.read())

In [3]:
df = pd.read_csv('emails.csv')
#data[10005]['header_info']

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517401 entries, 0 to 517400
Data columns (total 2 columns):
file       517401 non-null object
message    517401 non-null object
dtypes: object(2)
memory usage: 7.9+ MB


In [5]:
# Dictionary for abbreviated names/UID to e-mail address mapping
abrv_names = {}
for row in df['file']:
    name = row.split('/')[0]
    if not name in abrv_names.keys():
        abrv_names[name] = "Temp"

In [6]:
print ("Total unique abbreviated name in file: " + str(len(abrv_names.keys())))
for name in abrv_names.keys():
    print (name)

Total unique abbreviated name in file: 150
allen-p
arnold-j
arora-h
badeer-r
bailey-s
bass-e
baughman-d
beck-s
benson-r
blair-l
brawner-s
buy-r
campbell-l
carson-m
cash-m
causholli-m
corman-s
crandell-s
cuilla-m
dasovich-j
davis-d
dean-c
delainey-d
derrick-j
dickson-s
donoho-l
donohoe-t
dorland-c
ermis-f
farmer-d
fischer-m
forney-j
fossum-d
gang-l
gay-r
geaccone-t
germany-c
gilbertsmith-d
giron-d
griffith-j
grigsby-m
guzman-m
haedicke-m
hain-m
harris-s
hayslett-r
heard-m
hendrickson-s
hernandez-j
hodge-j
holst-k
horton-s
hyatt-k
hyvl-d
jones-t
kaminski-v
kean-s
keavey-p
keiser-k
king-j
kitchen-l
kuykendall-t
lavorato-j
lay-k
lenhart-m
lewis-a
linder-e
lokay-m
lokey-t
love-p
lucci-p
maggi-m
mann-k
martin-t
may-l
mccarty-d
mcconnell-m
mckay-b
mckay-j
mclaughlin-e
merriss-s
meyers-a
mims-thurston-p
motley-m
neal-s
nemec-g
panus-s
parks-j
pereira-s
perlingiere-d
phanis-s
pimenov-v
platter-p
presto-k
quenet-j
quigley-d
rapp-b
reitmeyer-j
richey-c
ring-a
ring-r
rodrique-r
rogers-b
ruscitti-k

In [7]:
max_sent = {}

In [8]:
# Build mapping using X-origin as source
# Find max sent from name
for row in df['message']:
    try:
        uid = row.split('\n')[13].split(' ')[1].lower()
        from_ = row.split('\n')[2].split('@')[0].split(' ')[1]

        # Only map existing uids
        if uid in abrv_names.keys():
            # Ignore one-off corner cases for the mapping purposes only
            if from_ != 'info' and \
                from_ != 'activetrader' and \
                from_ != 'trader' and \
                from_ != 'office-chairman' and \
                from_ != 'office.chairman' and \
                from_ != 'customer-service' and \
                from_ != 'enron.announcements' and \
                from_ != 'doctor' and \
                from_.find('..') == -1:
                    
                if not uid in max_sent.keys():
                    max_sent[uid] = {}

                if not from_ in max_sent[uid].keys():
                    max_sent[uid][from_] = 1
                else:
                    max_sent[uid][from_] += 1

    except Exception as e:
        None

In [9]:
 # Associate uid and job title
for uid in abrv_names.keys():
    max_count = 0
    for from_ in max_sent[uid].keys():
        if max_sent[uid][from_] > max_count:
            abrv_names[uid] = from_
            max_count = max_sent[uid][from_]

In [10]:
# Parse Job Titles Dataset
jt_dict = {}
jt_na_list = []
with open('enron_data.html', 'r') as file_:
    for line in file_.readlines():
        if line != '':
            # ID UID FName LName Rank
            details = line.split()
            uid = details[1]
            rank = ' '.join(details[4:])
            if rank != 'N/A' and rank != 'South N/A':
                jt_dict[uid] = rank
            else:
                jt_na_list.append(uid)

In [11]:
# CEO = 0
# C-Suite = 0
c_count = 0
# President = 1
p_count = 0
# VP = 2
vp_count = 0
# Director = 3
d_count = 0
# IHL = 4
ihl_count = 0
# Manager = 5
m_count = 0
# Trader / Specialist / Employee = 6
tse_count = 0

# UID -> Rank
employee_rank = {}

for rank in jt_dict.keys():
    raw_rank = jt_dict[rank]
    if raw_rank.find('CEO') != -1:
        #print (jt_dict[rank])
        #print (rank)
        employee_rank[rank] = 0
        c_count += 1
    elif raw_rank.find('Chief') != -1:
        #print (jt_dict[rank])
        #print (rank)
        employee_rank[rank] = 0
        c_count += 1
    elif raw_rank.find('Vice') != -1:
        #print (jt_dict[rank])
        #print (rank)
        employee_rank[rank] = 2
        vp_count += 1
    elif raw_rank.find('President') != -1:
        #print (jt_dict[rank])
        #print (rank)
        employee_rank[rank] = 1
        p_count += 1
    elif raw_rank.find('Director') != -1:
        #print (jt_dict[rank])
        #print (rank)
        employee_rank[rank] = 3
        d_count += 1
    elif raw_rank.find('Lawyer') != -1:
        #print (jt_dict[rank])
        #print (rank)
        employee_rank[rank] = 4
        ihl_count += 1
    elif raw_rank.find('Manager') != -1:
        #print (jt_dict[rank])
        #print (rank)
        employee_rank[rank] = 5
        m_count += 1
    else:
        #print (jt_dict[rank])
        #print (rank)
        employee_rank[rank] = 6
        tse_count += 1        

In [12]:
print ('C Level: ' + str(c_count))
print ('Presidents: ' + str(p_count))
print ('VPs: ' + str(vp_count))
print ('Directors: ' + str(d_count))
print ('In-House Lawyers: ' + str(ihl_count))
print ('Managers: ' + str(m_count))
print ('T/S/Es: ' + str(tse_count))

C Level: 8
Presidents: 4
VPs: 17
Directors: 16
In-House Lawyers: 3
Managers: 9
T/S/Es: 51


In [13]:
reverse_abrv_names = {v : k for k, v in abrv_names.items()}
reverse_abrv_names

{'phillip.allen': 'allen-p',
 'john.arnold': 'arnold-j',
 'harry.arora': 'arora-h',
 'robert.badeer': 'badeer-r',
 'susan.bailey': 'bailey-s',
 'eric.bass': 'bass-e',
 'don.baughman': 'baughman-d',
 'sally.beck': 'beck-s',
 'robert.benson': 'benson-r',
 'lynn.blair': 'blair-l',
 'sandra.brawner': 'brawner-s',
 'rick.buy': 'buy-r',
 'larry.campbell': 'campbell-l',
 'mike.carson': 'carson-m',
 'michelle.cash': 'cash-m',
 'monika.causholli': 'causholli-m',
 'shelley.corman': 'corman-s',
 'sean.crandall': 'crandell-s',
 'martin.cuilla': 'cuilla-m',
 'jeff.dasovich': 'dasovich-j',
 'dana.davis': 'davis-d',
 'dgagliardi': 'dean-c',
 'david.delainey': 'delainey-d',
 'james.derrick': 'derrick-j',
 'stacy.dickson': 'dickson-s',
 'lindy.donoho': 'donoho-l',
 'tom.donohoe': 'donohoe-t',
 'chris.dorland': 'dorland-c',
 'noreply': 'lucci-p',
 'daren.farmer': 'farmer-d',
 'jeff.duff': 'fischer-m',
 'john.forney': 'forney-j',
 'drew.fossum': 'fossum-d',
 'oasis502': 'gang-l',
 'randall.gay': 'gay-r',

In [14]:
# from_:to_ {message}
#unique to_ for testing
#to_set = set([])

from_to_ = {}
for index, row in df.iterrows():
    row = row['message']
    from_ = row.split('\n')[2].split('@')[0].split(' ')[1]
    to_ = row.split('\n')[3].split('@')[0].split(' ')[1]
    
    #to_set.add(to_)
    if to_ in reverse_abrv_names.keys() and from_ in reverse_abrv_names.keys() and to_ != from_:
        f_uid = reverse_abrv_names[from_]
        t_uid = reverse_abrv_names[to_]
        if f_uid in employee_rank.keys() and t_uid in employee_rank.keys():
            try:
                subject = row.split('\n')[4].split(' ')[1]
            except:
                subject = row.split('\n')[4]

            message = ''.join(row.split('\n')[15:])
            if message.find('Forwarded') != -1:
                message = 'Forwarded'

            rlshp = from_ + ':' + to_
            if rlshp in from_to_.keys():
                from_to_[rlshp] += ' ' + subject + ' ' + message
            else:
                from_to_[rlshp] = subject + ' ' + message        
    

In [15]:
print ("# Relationships: " + str(len(from_to_.keys())))

# Relationships: 719


In [16]:
# Convert to label -> text
pre_prep_format = []
u_count = 0
nu_count = 0

for key in from_to_.keys():
    from_, to_ = key.split(':')
    f_uid = reverse_abrv_names[from_]
    f_rank = employee_rank[f_uid]
    t_uid = reverse_abrv_names[to_]
    t_rank = employee_rank[t_uid]
    
    if f_rank > t_rank:
        pre_prep_format.append(('upward', from_to_[key]))
        u_count += 1
    elif f_rank < t_rank:
        nu_count += 1
        pre_prep_format.append(('not-upward', from_to_[key]))
    #else:
    #    pre_prep_format.append(('neutral', from_to_[key]))

In [17]:
print ("Labelled messages: " + str(len(pre_prep_format)))
print ("Upward messages: " + str(u_count))
print ("Not-upward messages: " + str(nu_count))

Labelled messages: 494
Upward messages: 256
Not-upward messages: 238


In [51]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nic\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nic\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [53]:
import string
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re

In [56]:
stop = set(stopwords.words('english'))

post_prep_format = []

for label, message in pre_prep_format:
    processed_message = message.lower()
    
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(processed_message)
    
    filtered_words = filter(lambda token: token not in stopwords.words('english'), tokens)
    
    # Return
    post_prep_format.append((label, ' '.join(filtered_words)))

In [21]:
#print (pre_prep_format[50])

In [22]:
#print (post_prep_format[50])

In [57]:
df = pd.DataFrame(post_prep_format, columns=['label', 'message'])

In [58]:
df_train = df[:round(df.shape[0] *.7)]
df_test = df[round(df.shape[0] *.7):]

In [59]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import numpy as np

tfidf_transformer = TfidfTransformer()
baseline_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB()),
])

In [60]:
baseline_clf.fit(df_train['message'], df_train['label'])

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [61]:
predicted = baseline_clf.predict(df_test['message'])
print ("Initial baseline accuracy: " + str(np.mean(predicted == df_test['label'])))

Initial baseline accuracy: 0.5202702702702703


In [63]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

In [64]:
model = Sequential()

In [65]:
df_train['message'].shape

(346,)

In [66]:
word_count = set([])

for message in df_train['message']:
    word_count.add(message)

In [68]:
len(word_count)

341

In [39]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path

In [70]:
num_labels = 2
vocab_size = 25000
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(df_train['message'])
 
x_train = tokenizer.texts_to_matrix(df_train['message'], mode='tfidf')
x_test = tokenizer.texts_to_matrix(df_test['message'], mode='tfidf')
 
encoder = LabelBinarizer()
encoder.fit(df_train['label'])
y_train = encoder.transform(df_train['label'])
y_test = encoder.transform(df_test['label'])

In [71]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 512)               12800512  
_________________________________________________________________
activation_19 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 512)               262656    
_________________________________________________________________
activation_20 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 2)                 1026      
__________